In [ ]:
%pip install selenium webdriver-manager beautifulsoup4 lxml pandas

In [ ]:
"""
Juniper Networks ACX Series EOL Table Scraper for Databricks
Class-based implementation with proper separation of concerns
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from typing import List, Optional


class JuniperEOLScraper:
    """
    Web scraper class for extracting End-of-Life tables from Juniper Networks website
    """
    
    def __init__(self, url: str, wait_time: int = 10, headless: bool = True):
        """
        Initialize the scraper with configuration
        
        Args:
            url: Target URL to scrape
            wait_time: Seconds to wait for page load
            headless: Run browser in headless mode
        """
        self.url = url
        self.wait_time = wait_time
        self.headless = headless
        self.driver = None
        self.tables = []
        self.dataframes = []
        
    def _setup_chrome_options(self) -> Options:
        """
        Configure Chrome options for Databricks environment
        
        Returns:
            Configured Chrome Options object
        """
        temp_dir = "/tmp/selenium_temp"
        os.makedirs(temp_dir, exist_ok=True)
        
        chrome_options = Options()
        
        if self.headless:
            chrome_options.add_argument("--headless=new")
        
        # Essential arguments for Databricks
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--disable-software-rasterizer")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-setuid-sandbox")
        
        # Critical: Set temp directories to /tmp
        chrome_options.add_argument(f"--user-data-dir={temp_dir}/user-data")
        chrome_options.add_argument(f"--data-path={temp_dir}/data-path")
        chrome_options.add_argument(f"--homedir={temp_dir}")
        chrome_options.add_argument(f"--disk-cache-dir={temp_dir}/cache")
        
        chrome_options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0 Safari/537.36"
        )
        
        return chrome_options
    
    def _initialize_driver(self):
        """
        Initialize Selenium WebDriver with configured options
        """
        chrome_options = self._setup_chrome_options()
        service = Service(executable_path="/usr/local/bin/chromedriver")
        
        print("Initializing Chrome WebDriver...")
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
    
    def _load_page(self):
        """
        Load the target webpage and wait for content
        """
        print(f"Loading page: {self.url}")
        self.driver.get(self.url)
        
        print(f"Waiting {self.wait_time} seconds for JavaScript to execute...")
        time.sleep(self.wait_time)
        
        # Wait for tables to appear
        try:
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "table"))
            )
            print("✓ Tables detected on page")
        except Exception as e:
            print(f"⚠ Warning: Timeout waiting for tables - {str(e)}")
            print("Proceeding with available content...")
    
    def _extract_tables_html(self) -> List:
        """
        Extract table HTML elements from the page
        
        Returns:
            List of BeautifulSoup table elements
        """
        html = self.driver.page_source
        soup = BeautifulSoup(html, "lxml")
        
        table_nodes = soup.find_all("table")
        print(f"Found {len(table_nodes)} table(s)")
        
        return table_nodes
    
    def _extract_hyperlinks(self, table) -> List[Optional[str]]:
        """
        Extract hyperlinks from the first column of a table
        
        Args:
            table: BeautifulSoup table element
            
        Returns:
            List of URLs or None values
        """
        links = []
        rows = table.select("tr")[1:]  # Skip header row
        
        for tr in rows:
            td = tr.find("td")
            a = td.find("a") if td else None
            links.append(a["href"] if a and a.has_attr("href") else None)
        
        return links
    
    @staticmethod
    def _drop_unnamed_cols(df: pd.DataFrame) -> pd.DataFrame:
        """
        Remove unnamed columns from DataFrame
        
        Args:
            df: Input DataFrame
            
        Returns:
            DataFrame with unnamed columns removed
        """
        return df.loc[:, ~df.columns.astype(str).str.match(r"^Unnamed")]
    
    def scrape_tables(self) -> List[pd.DataFrame]:
        """
        Main scraping method - orchestrates the entire scraping process
        
        Returns:
            List of pandas DataFrames containing scraped table data
        """
        try:
            # Initialize driver
            self._initialize_driver()
            
            # Load page
            self._load_page()
            
            # Extract table HTML
            table_nodes = self._extract_tables_html()
            
            if len(table_nodes) == 0:
                print("\n❌ No tables found on the page!")
                return []
            
            # Parse tables with pandas
            html = self.driver.page_source
            dfs = pd.read_html(html)
            
            # Process each table
            self.dataframes = []
            for i, (df, table) in enumerate(zip(dfs, table_nodes), start=1):
                print(f"\nProcessing Table {i}: {df.shape}")
                
                # Clean DataFrame
                df = self._drop_unnamed_cols(df)
                
                # Extract hyperlinks
                links = self._extract_hyperlinks(table)
                
                # Align link list length with df length
                if len(links) != len(df):
                    links = (links + [None] * len(df))[:len(df)]
                
                # Insert URL column after first column
                if len(df.columns) > 0:
                    first_col = df.columns[0]
                    df.insert(1, f"{first_col}_url", links)
                
                self.dataframes.append(df)
            
            print(f"\n✓ Successfully extracted {len(self.dataframes)} table(s)")
            return self.dataframes
            
        except Exception as e:
            print(f"\n❌ Error during scraping: {str(e)}")
            import traceback
            traceback.print_exc()
            return []
            
        finally:
            self._cleanup()
    
    def _cleanup(self):
        """
        Clean up resources and close browser
        """
        if self.driver:
            self.driver.quit()
            print("\n✓ Browser closed")
    
    def display_tables(self):
        """
        Display all extracted tables using Databricks display()
        """
        if not self.dataframes:
            print("No tables to display. Run scrape_tables() first.")
            return
        
        print("="*70)
        print("DATA PREVIEW")
        print("="*70 + "\n")
        
        for idx, df in enumerate(self.dataframes, start=1):
            print(f"\n--- Table {idx}: {df.shape} ---")
            display(df.head(10))
    
    def save_to_csv(self, output_path: str = "/dbfs/tmp", prefix: str = "juniper_acx_table"):
        """
        Save all extracted tables to CSV files
        
        Args:
            output_path: Directory path to save CSV files
            prefix: Filename prefix for CSV files
        """
        if not self.dataframes:
            print("No tables to save. Run scrape_tables() first.")
            return
        
        # Ensure output directory exists
        os.makedirs(output_path, exist_ok=True)
        
        saved_files = []
        for idx, df in enumerate(self.dataframes, start=1):
            filename = f"{prefix}_{idx}.csv"
            filepath = os.path.join(output_path, filename)
            
            df.to_csv(filepath, index=False)
            saved_files.append(filepath)
            print(f"✓ Saved Table {idx}: {filepath} ({df.shape[0]} rows × {df.shape[1]} columns)")
        
        return saved_files
    
    def get_dataframes(self) -> List[pd.DataFrame]:
        """
        Get the list of extracted DataFrames
        
        Returns:
            List of pandas DataFrames
        """
        return self.dataframes
    
    def get_summary(self) -> dict:
        """
        Get summary statistics of extracted tables
        
        Returns:
            Dictionary with summary information
        """
        if not self.dataframes:
            return {"status": "No tables extracted"}
        
        summary = {
            "total_tables": len(self.dataframes),
            "tables": []
        }
        
        for idx, df in enumerate(self.dataframes, start=1):
            summary["tables"].append({
                "table_number": idx,
                "shape": df.shape,
                "columns": list(df.columns),
                "rows": len(df),
                "columns_count": len(df.columns)
            })
        
        return summary


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("JUNIPER NETWORKS ACX SERIES EOL TABLE SCRAPER - DATABRICKS")
    print("="*70 + "\n")
    
    # Configuration
    URL = "https://support.juniper.net/support/eol/product/acx_series/"
    OUTPUT_PATH = "/dbfs/tmp"
    FILE_PREFIX = "juniper_acx_series_table"
    
    # Initialize scraper
    scraper = JuniperEOLScraper(
        url=URL,
        wait_time=10,
        headless=True
    )
    
    # Scrape tables
    print("Starting scraping process...\n")
    tables = scraper.scrape_tables()
    
    if tables:
        # Display summary
        print(f"\n{'='*70}")
        print("✓ EXTRACTION SUCCESSFUL!")
        print("="*70 + "\n")
        
        summary = scraper.get_summary()
        print(f"Total tables extracted: {summary['total_tables']}\n")
        
        for table_info in summary['tables']:
            print(f"Table {table_info['table_number']}: "
                  f"{table_info['rows']} rows × {table_info['columns_count']} columns")
            print(f"  Columns: {', '.join(table_info['columns'][:5])}...")
            print()
        
        # Save to CSV
        print(f"\n{'='*70}")
        print("SAVING TO CSV FILES")
        print("="*70 + "\n")
        
        saved_files = scraper.save_to_csv(
            output_path=OUTPUT_PATH,
            prefix=FILE_PREFIX
        )
        
        # Display tables
        scraper.display_tables()
        
        print(f"\n{'='*70}")
        print("✓ PROCESS COMPLETED SUCCESSFULLY")
        print("="*70)
        
    else:
        print("\n❌ EXTRACTION FAILED - No tables found")
        print("\nTroubleshooting tips:")
        print("1. Verify the URL is accessible")
        print("2. Increase wait_time in scraper initialization")
        print("3. Check if Chrome and ChromeDriver are properly installed")
        print("4. Run with headless=False to debug visually")
